In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os, shutil
import cv2
import glob as gb
import pandas as pd
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings("ignore")

In [8]:
# create folder data with zipped images from kaggle!!!

path_file_cat = 'data/PetImages/Cat'
path_file_dog = 'data/PetImages/Dog'

base_dir = 'data/PetImages/Train'

# Train folder
Train_dir = os.path.join(base_dir, 'Train')

cat_train_dir = os.path.join(Train_dir, 'cat')
dog_train_dir = os.path.join(Train_dir, 'dog')

# Test Folder
Test_dir = os.path.join(base_dir, 'Test')

cat_test_dir = os.path.join(Test_dir, 'cat')
dog_test_dir = os.path.join(Test_dir, 'dog')

# Validation Folder
valid_dir = os.path.join(base_dir, 'validation')
cat_val_dir = os.path.join(valid_dir, 'cat')
dog_val_dir = os.path.join(valid_dir, 'dog')

In [9]:
# Creating folders
os.mkdir(base_dir)

# Train folder
os.mkdir(Train_dir)
os.mkdir(cat_train_dir)
os.mkdir(dog_train_dir)

# Test Folder
os.mkdir(Test_dir)
os.mkdir(cat_test_dir)
os.mkdir(dog_test_dir)

# validation folder
os.mkdir(valid_dir)
os.mkdir(cat_val_dir)
os.mkdir(dog_val_dir)

In [ ]:
train_list_cat_file = [f'{i}.jpg' for i in range(6500)]
train_list_dog_file = [f'{i}.jpg' for i in range(6500)]

test_list_cat_file = [f'{i}.jpg' for i in range(6500, 9500)]
test_list_dog_file = [f'{i}.jpg' for i in range(6500, 9500)]

val_list_cat_file = [f'{i}.jpg' for i in range(9500, 12500)]
val_list_dog_file = [f'{i}.jpg' for i in range(9500, 12500)]

In [ ]:
# organaize cat and dog in train folders
# cat
for filename in train_list_cat_file:
    src = os.path.join(path_file_cat, filename)
    dst = os.path.join(cat_train_dir, filename)
    shutil.copyfile(src, dst)
    
# dog
for filename in train_list_dog_file:
    src = os.path.join(path_file_dog, filename)
    dst = os.path.join(dog_train_dir, filename)
    shutil.copyfile(src, dst)
    
# organaize cat and dog in test folders
# cat
for filename in test_list_cat_file:
    src = os.path.join(path_file_cat, filename)
    dst = os.path.join(cat_test_dir, filename)
    shutil.copyfile(src, dst)
    
# dog
for filename in test_list_dog_file:
    src = os.path.join(path_file_dog, filename)
    dst = os.path.join(dog_test_dir, filename)
    shutil.copyfile(src, dst)
    
# organaize cat and dog in validation folders
# cat
for filename in val_list_cat_file:
    src = os.path.join(path_file_cat, filename)
    dst = os.path.join(cat_val_dir, filename)
    shutil.copyfile(src, dst)
    
# dog
for filename in val_list_dog_file:
    src = os.path.join(path_file_dog, filename)
    dst = os.path.join(dog_val_dir, filename)
    shutil.copyfile(src, dst)

In [ ]:
print("number of cat img in train folder: ", len(os.listdir(cat_train_dir)))
print("number of dog img in train folder: ", len(os.listdir(dog_train_dir)))
print("number of cat img in test folder: ", len(os.listdir(cat_test_dir)))
print("number of dog img in test folder: ", len(os.listdir(dog_test_dir)))
print("number of cat img in validation folder: ", len(os.listdir(cat_val_dir)))
print("number of dog img in validation folder: ", len(os.listdir(dog_val_dir)))

number of cat img in train folder:  6500
number of dog img in train folder:  6500
number of cat img in test folder:  3000
number of dog img in test folder:  3000
number of cat img in validation folder:  3000
number of dog img in validation folder:  3000


In [22]:

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Parametry
IMG_SIZE = 150
CAT_DIR = "C:\\Users\\Szymon\\git_is\projekt\\projekt_dl\\data\\PetImages\\Train\\Train/dog"
DOG_DIR = "C:\\Users\\Szymon\\git_is\projekt\\projekt_dl\\data\\PetImages\\Train\\Train/cat"

# Wczytywanie obrazów
def load_images(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img = cv2.imread(os.path.join(directory, filename))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(label)
    return images, labels

cats, cat_labels = load_images(CAT_DIR, 0)  # 0 dla kota
dogs, dog_labels = load_images(DOG_DIR, 1)  # 1 dla psa

X = np.array(cats + dogs)
y = np.array(cat_labels + dog_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
263/325 [=======================>......] - ETA: 2:49 - loss: 49.6545 - accuracy: 0.5329

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Wybierz obrazek do wizualizacji
img = X_test[0].reshape(1, IMG_SIZE, IMG_SIZE, 3)

# Pobierz wyjścia wszystkich warstw
layer_outputs = [layer.output for layer in model.layers]

# Utwórz model, który zwraca te wyjścia dla danego wejścia
activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)

# Pobierz aktywacje
activations = activation_model.predict(img)

# Wizualizuj aktywacje pierwszej warstwy konwolucyjnej
first_layer_activation = activations[0]
plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')
plt.show()

In [ ]:

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    plt.xlabel("Pies" if y_train[i] == 1 else "Kot")
plt.show()

model sieci konwolucyjnej (CNN) 

In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Inicjalizacja modelu sekwencyjnego
model = Sequential()

# Dodanie warstw do modelu
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Kompilacja modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Opisz każdą warstwę używając komentarzy w kodzie


In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

# Wejście modelu
input_img = Input(shape=(64, 64, 3))

# Pierwsza warstwa konwolucyjna
x = Conv2D(32, (3, 3), activation='relu')(input_img)
x = MaxPooling2D((2, 2))(x)

# Druga warstwa konwolucyjna
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

# Trzecia warstwa konwolucyjna
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

# Spłaszczenie i pełna połączona warstwa
x = Flatten()(x)
x = Dense(512, activation='relu')(x)

# Dropout
x = Dropout(0.5)(x)

# Warstwa wyjściowa
output = Dense(1, activation='sigmoid')(x)

# Tworzenie modelu
model = Model(inputs=input_img, outputs=output)

# Kompilacja modelu
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Podsumowanie modelu
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_21 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_23 (Conv2D)          (None, 12, 12, 128)       7385

In [27]:
# Przygotowanie danych (przykładowe ścieżki do katalogów)
train_directory = 'C:\\Users\\Szymon\\git_is\projekt\\projekt_dl\\data\\PetImages\\Train\\Train'
validation_directory = 'C:\\Users\\Szymon\\git_is\projekt\\projekt_dl\\data\\PetImages\\Train\\Test'

# Augmentacja danych dla zestawu treningowego
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augmentacja danych jest zazwyczaj stosowana tylko dla zestawu treningowego.
validation_datagen = ImageDataGenerator(rescale=1./255)

# Przepływ danych treningowych i walidacyjnych
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_directory,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', min_lr=0.00001)
]


Found 13000 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.


In [21]:
# Uczenie modelu
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size,
    callbacks=callbacks
)

Epoch 1/10
139/406 [=========>....................] - ETA: 1:22 - loss: 0.6712 - accuracy: 0.5800

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000024F7BEFD210>
Traceback (most recent call last):

  File "c:\Users\Szymon\Anaconda3\Lib\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "c:\Users\Szymon\Anaconda3\Lib\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\Szymon\Anaconda3\Lib\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "c:\Users\Szymon\Anaconda3\Lib\site-packages\keras\src\engine\data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():
